# Suspect of too low accuracy in Lesion model
Vocabulary: 
- boardcasting: when adding tensor with different dimension, for example input_p (dim = [batch_size, 250]) and bias_p (dim = [1, 250]), bias_p will automatically "boardcast" to (batch_size, 250) during the elementwise addition

When input information have not reach a layer, it will assume the first axis (batch_size) = 1 
however, the batch_size won't matter... since tf.matmul batch_axis is independent with each other...  


In [ ]:
import tensorflow as tf
import pandas as pd
import altair as alt
import os, random
import meta, modeling, data_wrangling, evaluate, troubleshooting
from importlib import reload

In [ ]:
reload(troubleshooting)
d = troubleshooting.Diagnosis(code_name='Refrac_5M_fix')
d.eval(testset_name='train_r100', task='triangle', epoch=400)
random.sample(list(d.all_correct_words.values()), 10)


# Correct word

In [ ]:
d.set_target_word('sick')

In [ ]:
d.plot_one_layer('pho')

In [ ]:
d.plot_one_layer('sem')

- On target is OS driven, as long as it is intact, it shouldn't have problem
- Off target is PS drive, lesioning PS may harm accuracy?

- also note that, due to time averaged input, the activation is

- Pho layer is solid, lesion will not change the activation too much

# How about in the incorrect words

In [ ]:
d.all_incorrect_words

# First we examine an incorrect wor in semantic

In [ ]:
d.set_target_word('fart')


In [ ]:
d.plot_one_layer('pho')

In [ ]:
d.plot_one_layer('sem')

- PS is dominating the input
- CS generally surpressing activation

# Incorrect word in PHO

In [ ]:

d.set_target_word('breadths')


In [ ]:

d.plot_one_layer('sem')


In [ ]:

d.plot_one_layer('pho')


In [ ]:
def mean_unit_input(tensor, mask=None):
    """Mask with output target, reduce sum on items (axis1), mean on units (axis2)"""
    if mask is not None:
        tensor = tf.multiply(tensor, mask)
    return tf.reduce_mean(tf.reduce_mean(tensor, axis=-1), axis=-1)

In [ ]:

results_dict = {k: mean_unit_input(globals()[k], train100['sem']) for k in ('os', 'ps', 'cs', 'ss')}
results_dict['op'] = mean_unit_input(op, train100['pho'])
dol_df = pd.DataFrame.from_dict(results_dict)
dol_df.plot(title="one target")

In [ ]:
results_dict = {k: mean_unit_input(globals()[k], 1-train100['sem']) for k in ('os', 'ps', 'cs', 'ss')}
results_dict['op'] = mean_unit_input(op, 1-train100['pho'])
dol_df = pd.DataFrame.from_dict(results_dict)
dol_df.plot(title="zero target")

In [ ]:
bias_s = [w for w in model.weights if w.name == "my_model/bias_s:0"]
bias_p = [w for w in model.weights if w.name == "my_model/bias_p:0"]
print(f"mean bias S: {tf.reduce_mean(bias_s)}; P: {tf.reduce_mean(bias_p)}")